In [1]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes
import requests

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline

from bs4 import BeautifulSoup
from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

import bs4 as bs
import urllib.request

# geocoder
!conda install -c conda-forge geocoder --yes 
import geocoder

print('Libraries imported.')

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20180922210842-0000
Solving environment: done

## Package Plan ##

  environment location: /opt/ibm/conda/miniconda3

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_0          52 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    ca-certificates-2018.8.24  |       ha4d7672_0         136 KB  conda-forge
    ratelim-0.1.6              |           py35_0           5 KB  conda-forge
    openssl-1.0.2p             |       h470a237_0         3.5 MB  conda-forge
    orderedset-2.0             |           py35_0         685 KB  conda-forge
    conda-4.5.11               |           py35_0         636 KB  conda-forge
    ----------------------------------------------------------

In [2]:
# Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
source = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').read()
soup = bs.BeautifulSoup(source,'html.parser')

table = soup.find('table')
table_rows = table.find_all('tr')

l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        l.append(row)

In [3]:
df = pd.DataFrame(l, columns=["PostalCode", "Borough", "Neighbourhood"])
df.head(10)

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [4]:
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
df = df[df.Borough != 'Not assigned']
df.head(10)

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [5]:
# More than one neighborhood can exist in one postal code area.
df = df.groupby(['PostalCode', 'Borough']).agg(', '.join)
df = df.reset_index()
df.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood]], Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [6]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df.loc[df['Neighbourhood']=='Not assigned', ['Neighbourhood']] = 'Queen\'s Park'
df.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood]], Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


What I have done so far on the data cleaning above?
1. Import and scrap wikipage using BeautifulSoup package.
2. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.
3. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
4. The rows of same PostalCode and Borough will be combined into one row with the neighborhoods separated with a comma.
5. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [9]:
# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
df.shape

(103, 3)

<b>Project Part 2: Getting the latitude and the longitude coordinates of each neighborhood.</b><br><br>1. Leveraging on the geocoder API.<br>2. Update the coordinates accordingly in the dataframe.<br>3. Display them out accordingly to the layout in the instruction.

In [23]:
df['Latitude'] = ''
df['Longitude'] = ''

# loop through the postal code to find out the latitude and longitude
for index, data in df.iterrows():
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(data['PostalCode']))
        lat_lng_coords = g.latlng
    data['Latitude'] = lat_lng_coords[0]
    data['Longitude'] = lat_lng_coords[1]
    print ('PostalCode:', data['PostalCode'], 'Latitude:', data['Latitude'], 'Longitude:', data['Longitude'])

PostalCode: M1B Latitude: 43.8066863 Longitude: -79.1943534
PostalCode: M1C Latitude: 43.78453510000001 Longitude: -79.1604971
PostalCode: M1E Latitude: 43.7635726 Longitude: -79.1887115
PostalCode: M1G Latitude: 43.7709921 Longitude: -79.2169174
PostalCode: M1H Latitude: 43.773136 Longitude: -79.23947609999999
PostalCode: M1J Latitude: 43.7447342 Longitude: -79.23947609999999
PostalCode: M1K Latitude: 43.7279292 Longitude: -79.2620294
PostalCode: M1L Latitude: 43.7111117 Longitude: -79.2845772
PostalCode: M1M Latitude: 43.716316 Longitude: -79.23947609999999
PostalCode: M1N Latitude: 43.692657 Longitude: -79.2648481
PostalCode: M1P Latitude: 43.7574096 Longitude: -79.273304
PostalCode: M1R Latitude: 43.7500715 Longitude: -79.2958491
PostalCode: M1S Latitude: 43.7942003 Longitude: -79.2620294
PostalCode: M1T Latitude: 43.7816375 Longitude: -79.3043021
PostalCode: M1V Latitude: 43.8152522 Longitude: -79.2845772
PostalCode: M1W Latitude: 43.7995252 Longitude: -79.3183887
PostalCode: M1X 

In [25]:
# print out the updated dataframe
df.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.8067,-79.1944
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.7845,-79.1605
2,M1E,Scarborough,"Guildwood]], Morningside, West Hill",43.7636,-79.1887
3,M1G,Scarborough,Woburn,43.771,-79.2169
4,M1H,Scarborough,Cedarbrae,43.7731,-79.2395
5,M1J,Scarborough,Scarborough Village,43.7447,-79.2395
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.7279,-79.262
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.7111,-79.2846
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.7163,-79.2395
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.6927,-79.2648


In [27]:
# double check on the dataframe
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


<b>Outcome:</b><br><br>For above, I have added the latitude and longtitude for each postal code in the dataframe.